# Assignment 1: Market Clearing (System Perspective)

### Import external libraries

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np 
from network import initialize_network, system_demand

## Step 0: Build a Relevant Case Study

Please select an electric power network from the following options:
1. IEEE 24-bus reliability test system: link.
2. IEEE reliability test system (2019 update): link.
3. IEEE power systems test cases (various cases with 14, 30, 57, 118, and 300 buses): link.
4. Case studies available in the open-source Julia platform PowerModels.jl: link.

You are also free to choose another case study. If some data is missing, please select reason-
able arbitrary values. For technical details on conventional generators and transmission lines, this link may be helpful (it corresponds to the IEEE 24-bus case study, but similar data can be used for other cases).

### IEEE 24-bus reliability test system

In [2]:
consumers, generators = initialize_network()
print(generators)
print(consumers)
print("Network initialized successfully.")

[Generator(unit=1, node=1), Generator(unit=2, node=2), Generator(unit=3, node=7), Generator(unit=4, node=13), Generator(unit=5, node=15), Generator(unit=6, node=15), Generator(unit=7, node=16), Generator(unit=8, node=18), Generator(unit=9, node=21), Generator(unit=10, node=22), Generator(unit=11, node=23), Generator(unit=12, node=23)]
[Consumer(load=1, node=1, share=0.038), Consumer(load=2, node=2, share=0.034), Consumer(load=3, node=3, share=0.063), Consumer(load=4, node=4, share=0.026), Consumer(load=5, node=5, share=0.025), Consumer(load=6, node=6, share=0.048), Consumer(load=7, node=7, share=0.044), Consumer(load=8, node=8, share=0.06), Consumer(load=9, node=9, share=0.061), Consumer(load=10, node=10, share=0.068), Consumer(load=11, node=13, share=0.093), Consumer(load=12, node=14, share=0.068), Consumer(load=13, node=15, share=0.111), Consumer(load=14, node=16, share=0.035), Consumer(load=15, node=18, share=0.117), Consumer(load=16, node=19, share=0.064), Consumer(load=17, node=20

### Additional Assumptions


*   Assume that the price bids of all producers are non-negative and equal to their marginal production cost. In particular, the production cost of renewable units is assumed to be zero. Additionally, these units offer their forecasted capacity, meaning their offer quantities vary over time.

*   For the bid price of price-elastic demands, use comparatively high values (relative to the generation cost of conventional units) to ensure that most demands are supplied. For inspiration, check the real bid price data in Nord Pool [link].
*   A potential source for wind power forecast data is available at this link (you may nor- malize the data to fit your case study). Another potential source for the renewable power generation data is renewables.ninja.
*   For transmission lines, you may assume a uniform reactance for all lines (e.g., 0.002 p.u., leading to a susceptance of 500 p.u.).

## Step 1: Copper-Plate, Single Hour

In Lecture 2, you learn how to develop a market-clearing optimization model for a copper-plate
power system (i.e., without modeling the transmission network) in a single-hour setting.
Please determine the following market-clearing outcomes:


### Qur Optimzation Problem


\begin{align}
\textrm{minimize} \quad
&13.32x_1 + 13.32x_2 + 20.7x_3 + 20.93x_4 + 26.11x_5 + 10.52x_6 \\
&\quad + 10.52x_7 + 6.02x_8 + 5.47x_9 + 0x_{10} + 10.52x_{11} + 10.89x_{12} \\
\textrm{subject to} \\
&0 \le  x_1 \le 152 \\
&0 \le x_1 \le 152 \\
&0 \le x_2 \le 152 \\
&0 \le x_3 \le 350 \\
&0 \le x_4 \le 591 \\
&0 \le x_5 \le 60 \\
&0 \le x_6 \le 155 \\
&0 \le x_7 \le 155 \\
&0 \le x_8 \le 400 \\
&0 \le x_9 \le 400 \\
&0 \le x_{10} \le 300 \\
&0 \le x_{11} \le 310 \\
&0 \le x_{12} \le 350 \\
& 2464.965 - \sum_{i=1}^{12} x_i = 0 \\
\end{align}


\begin{align}
\textrm{minimize} \quad 
&\sum_{i=1}^{12} C_i *x_i \\
\end{align}

\begin{align}
\textrm{where} \quad 
C_i = \textrm{offer price of generator i} \\
x_i =  \textrm{production of generator i} \\
\end{align}



### The market-clearing price under a uniform pricing scheme.

In [3]:
# define the demand time at 16:00
total_consumption_16 = system_demand[16]
print(f"Total consumption at 16:00 is {total_consumption_16} MW")
#check if all generators are there
id_Gnerators = []
for i in generators:
    id_Gnerators.append(i.unit_id)
print(id_Gnerators)

Total consumption at 16:00 is 2464.965 MW
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Define the supply


In [4]:
#Variable generators costs 
generator_cost = []
for i in generators:
    generator_cost.append(i.cost_energy)
print(generator_cost)

#variable demand bids prize
demand_bids = []
#consumption for each consumer
consumption = []
for i in consumers:
    demand_bids.append(i.price)
    consumption.append(i.share * total_consumption_16)
print(demand_bids)
print(consumption)

if sum(consumption) == total_consumption_16:
    print("Aggregation correct")

[13.32, 13.32, 20.7, 20.93, 26.11, 10.52, 10.52, 6.02, 5.47, 0, 10.52, 10.89]
[165.0, 155.0, 175.0, 150.0, 145.0, 170.0, 160.0, 180.0, 185.0, 190.0, 210.0, 195.0, 220.0, 158.0, 215.0, 178.0, 162.0]
[93.66867, 83.80881000000001, 155.292795, 64.08909, 61.62412500000001, 118.31832000000001, 108.45846, 147.8979, 150.362865, 167.61762000000002, 229.241745, 167.61762000000002, 273.61111500000004, 86.27377500000001, 288.400905, 157.75776000000002, 110.92342500000001]
Aggregation correct


Define Generator capacity:

In [5]:
#Generators capacity
generator_capacity = []
for i in generators:
    generator_capacity.append(i.p_max)
print(generator_capacity)

[152, 152, 350, 591, 60, 155, 155, 400, 400, 300, 310, 350]


Everytime you use Gurobi, you will need to import the package ```gurobipy```. The specific module ```GRB``` is commonly imported separately, as it is used frequently. 

In [22]:
#create the optimization model
model = gp.Model("Economic_Dispatch_Model")

In [51]:
production_variables = [
    model.addVar(lb=0, ub = v , vtype=GRB.CONTINUOUS, name=f"p_{i}")
    for i,v in enumerate(generator_capacity)
]

consumption_variables = [
    model.addVar(lb=0, ub = v , vtype=GRB.CONTINUOUS, name=f"p_{i}")
    for i,v in enumerate(consumption)
]

In [63]:
#balance_constraint = model.addConstr(gp.quicksum(production_variables[i] for i,v in enumerate(generator_capacity)) == gp.quicksum(consumption_variables[i] for i,v in enumerate(consumption)), name="balance_constraint")
balance_constraint = model.addConstr(gp.quicksum(production_variables[i] for i,v in enumerate(generator_capacity)) == total_consumption_16, name="balance_constraint")



In [73]:
model.setObjective(
    gp.quicksum(consumption[j] * demand_bids[j] for j in range(len(consumption_variables)))
    - gp.quicksum(generator_cost[i] * production_variables[i] for i in range(len(production_variables))),
    GRB.MINIMIZE
)
#model.setObjective(
    #gp.quicksum(generator_cost[i] * production_variables[i] for i in range(len(production_variables))),
    #GRB.MINIMIZE
#)

In [74]:
model.optimize()

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[rosetta2] - Darwin 25.2.0 25C56)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 61 rows, 152 columns and 217 nonzeros (Min)
Model has 11 linear objective coefficients and an objective constant of 460632.93948
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [6e+01, 6e+02]
  RHS range        [6e+01, 2e+03]

LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0914763e+05   1.829956e+02   0.000000e+00      0s
       1    4.2392494e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.239249402e+05


In [75]:
#check status and print results 
if model.status == GRB.OPTIMAL:
    optimal_objective = model.objVal
    optimal_production_variables = [production_variables[i].x for i in range(len(generators))]
    optimal_consumption_variables = [consumption_variables[i].x for i in range(len(consumers))]
    balance_dual = balance_constraint.Pi
    capacity_duals = [capacity_constraints[i].Pi for i in range(len(generators))]

print (f"Optimal objective value: {optimal_objective}")

for index, optimal in enumerate(optimal_production_variables):
    print(f"Optimal production for Generator {id_Gnerators[index]}: {optimal} MW")

for index, optimal in enumerate(optimal_consumption_variables):
    print(f"Optimal demand for Consumer {index}: {optimal} MW")

print(f"Dual variable for balance constraint: {balance_dual}")

for index, dual in enumerate(capacity_duals):
    print(f"Dual variable for capacity constraint of Generator {id_Gnerators[index]}: {dual}")

else:
    print(f"optimization of model {model.ModelName} was not successful. Status code: {model.status}")

Optimal objective value: 423924.94018000003
Optimal production for Generator 1: 152.0 MW
Optimal production for Generator 2: 152.0 MW
Optimal production for Generator 3: 350.0 MW
Optimal production for Generator 4: 591.0 MW
Optimal production for Generator 5: 60.0 MW
Optimal production for Generator 6: 155.0 MW
Optimal production for Generator 7: 155.0 MW
Optimal production for Generator 8: 189.96500000000015 MW
Optimal production for Generator 9: 0.0 MW
Optimal production for Generator 10: 0.0 MW
Optimal production for Generator 11: 310.0 MW
Optimal production for Generator 12: 350.0 MW
Optimal demand for Consumer 0: 93.66867 MW
Optimal demand for Consumer 1: 83.80881000000001 MW
Optimal demand for Consumer 2: 155.292795 MW
Optimal demand for Consumer 3: 64.08909 MW
Optimal demand for Consumer 4: 61.62412500000001 MW
Optimal demand for Consumer 5: 118.31832000000001 MW
Optimal demand for Consumer 6: 108.45846 MW
Optimal demand for Consumer 7: 147.8979 MW
Optimal demand for Consumer 8:

In [76]:
print(f"The markets clearing price at 16:00 is: {balance_dual} $/MWh")

The markets clearing price at 16:00 is: -6.02 $/MWh


### The total operating cost and social welfare of the system.

In [59]:
print(f"Total cost of generation at 16:00 is: {optimal_objective} $")
print(f"Social welfare at 16:00 is: {total_consumption_16 * balance_dual - optimal_objective} $")

Total cost of generation at 16:00 is: 439770.78398 $
Social welfare at 16:00 is: -490795.55948 $


### The profit of each producer, including both conventional units and wind farms.

In [60]:
print(f"The production cost for each generator at 16:00 is:")
for i in range(len(generators)):
    cost = generator_cost[i] * optimal_production_variables[i]
    print(f"Generator {id_Gnerators[i]}: {cost} $")

The production cost for each generator at 16:00 is:
Generator 1: 2024.64 $
Generator 2: 2024.64 $
Generator 3: 1882.9755000000064 $
Generator 4: 0.0 $
Generator 5: 0.0 $
Generator 6: 1630.6 $
Generator 7: 1630.6 $
Generator 8: 2408.0 $
Generator 9: 2188.0 $
Generator 10: 0.0 $
Generator 11: 3261.2 $
Generator 12: 3811.5 $


### The utility of each demand, defined as:
$$

    \begin{align}
        \textrm{Utility = Power Consumption ×(Bid Price−Market-Clearing Price)}
    \end{align}

$$

In [62]:
for consumer in consumers:
    consuption = consumer.share * total_consumption_16
    bid_price = consumer.price
    utility = consuption * (bid_price - balance_dual)
    print(f"Consumer {consumer.load_id}: Consumption = {consuption} MW, Utility = {utility} $") 

Consumer 1: Consumption = 93.66867 MW, Utility = 17394.272019 $
Consumer 2: Consumption = 83.80881000000001 MW, Utility = 14725.207917 $
Consumer 3: Consumption = 155.292795 MW, Utility = 30390.7999815 $
Consumer 4: Consumption = 64.08909 MW, Utility = 10940.007662999999 $
Consumer 5: Consumption = 61.62412500000001 MW, Utility = 10211.1175125 $
Consumer 6: Consumption = 118.31832000000001 MW, Utility = 22563.303624 $
Consumer 7: Consumption = 108.45846 MW, Utility = 19598.443722 $
Consumer 8: Consumption = 147.8979 MW, Utility = 29683.108529999998 $
Consumer 9: Consumption = 150.362865 MW, Utility = 30929.6413305 $
Consumer 10: Consumption = 167.61762000000002 MW, Utility = 35317.032534 $
Consumer 11: Consumption = 229.241745 MW, Utility = 52886.0705715 $
Consumer 12: Consumption = 167.61762000000002 MW, Utility = 36155.120634 $
Consumer 13: Consumption = 273.61111500000004 MW, Utility = 65858.19538050001 $
Consumer 14: Consumption = 86.27377500000001 MW, Utility = 15417.123592500002 